In [1]:
from queue import Queue

In [2]:
class DAG:
    def __init__(self, adj_list):
        self.adj_list = adj_list
        self.reverse = {}
        self.edges = []
        self.nodes = set()

        for v in adj_list.keys():
            self.nodes.add(v)
            for u in adj_list[v]:
                self.nodes.add(u)
                self.edges.append((v,u))
                self.reverse.setdefault(u,[]).append(v)

    def neighbors(self, node):
        return set(self.adj_list.setdefault(node, []) + self.reverse.setdefault(node, []))


In [3]:
dag = DAG({1 : [2,3,4], 2 : [5,6], 3: [5]})
print(dag.neighbors(5))

{2, 3}


In [6]:
def k_core_decomposition(graph: DAG):
    '''
    Computes k-Core decomposition of the given DAG.

    Parameters
    ------
    graph : DAG
            directed acyclic graph to decompose
    
    Returns
    ------
    A dictionary, where keys are vertices and values are their core numbers.

    A k-core decomposition of a graph is a hierarchical decomposition, where a threshold k is set on a degree of
    each vertex, and those vertices which don't satisfy threshold are excluded from the process. Rinse and repeat - 
    start from lowest.

    Useful: degeneracy - highest k, such that H a subgraph of G is not empty.
    '''
    size = len(graph.nodes)

    max_deg = 0
    deg = [None]*size
    order = [None]*size
    number = {}

    for idx, node in enumerate(graph.nodes):
        deg[idx] = len(graph.neighbors(node))
        number[node] = idx
        order[idx] = node
        max_deg = max(max_deg, deg[idx])

    bin = [0]*(max_deg + 1)
    
    # Count vertices in each bin.
    for i in range(0, size):
        bin[deg[i]] += 1 

    # Sort by degree. Determine starting vertex in each bin.
    start = 0
    for i in range(0, max_deg + 1):
        tmp = bin[i]
        bin[i] = start
        start = start + tmp

    # Put sorted vertices into vert
    vert = [None]*size
    pos = [None]*size
    for i in range(0, size):
        pos[i] = bin[deg[i]]
        vert[pos[i]] = i
        bin[deg[i]] += 1

    # Recover starts.
    for i in reversed(range(1, max_deg + 1)):
        bin[i] = bin[i - 1]
    bin[0] = 0

    core = {}
    # Calculate core numbers.
    for i in range(0, size):
        curr_node = order[vert[i]]

        core[curr_node] = deg[vert[i]]

        for nb in graph.neighbors(curr_node):
            u_idx = number[nb]

            # Decrease vertex degrees.
            if deg[u_idx] > deg[vert[i]]:
                pu = pos[u_idx]
                pw = bin[deg[u_idx]]
                
                w_idx = vert[pw]
                if u_idx != w_idx:
                    pos[u_idx] = pw
                    pos[w_idx] = pu
                    vert[pu] = w_idx
                    vert[pw] = u_idx

                bin[deg[u_idx]] += 1
                deg[u_idx] -= 1

    return core


In [15]:
# K-Core decomposition tests.
# dag_k_3 = DAG({1: [2,3,4], 5:[2,3,4], 2: [3], 4: [3]})
# k_3 = k_core_decomposition(dag_k_3)
# k_3_expected = {1: 3, 2: 3, 4: 3, 5: 3, 3: 3}
# assert(k_3 == k_3_expected)

# dag_k_1_3 = DAG({1: [2, 3, 4, 6], 5:[2, 3, 4], 2: [3, 6, 7], 4: [3], 7: [8], 8: [9]})
# k_1_3 = k_core_decomposition(dag_k_1_3)
# k_1_3_expected = {9: 1, 8: 1, 7: 1, 6: 2, 4: 3, 5: 3, 1: 3, 2: 3, 3: 3}
# assert(k_1_3 == k_1_3_expected)

In [ ]:
def get_max_k(k_cores_decomposition: dict):
    return max(k_cores_decomposition.values())

def create_order(graph: DAG, k_cores_decomposition: dict, highest_in_center: bool = True):
    max_lvl = get_max_k(k_cores_decomposition) + 1
    order_temp = [[]]*max_lvl # +1 ?

    for n in graph.nodes:
        if highest_in_center:
            order_temp[max_lvl - k_core_decomposition[n]].append(n)
        else:
            order_temp[k_core_decomposition[n]].append(n)

    order = []
    # Remove empty levels.
    for lvl in order_temp:
        if len(lvl) != 0:
            order.append(lvl)
    
    return order

def get_level_assigments(order: list):
    res = {} 
    for idx, lvl in order:
        for n in lvl:
            res[n] = idx
    return res